<a href="https://colab.research.google.com/github/jerryjliu/llama_index/blob/main/docs/docs/examples/llm/predibase.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="在 Colab 中打开"/></a>


# Predibase

这个笔记本展示了如何在Llamaindex中使用Predibase托管的LLM。您可以将[Predibase](https://predibase.com)添加到现有的Llamaindex工作流中，以便：
1. 部署和查询预训练或自定义的开源LLM，无需麻烦
2. 将端到端的检索增强生成（RAG）系统实现运营化
3. 仅需几行代码即可对自己的LLM进行微调

## 入门指南
1. 在[这里](https://predibase.com/free-trial)免费注册Predibase账户
2. 创建一个账户
3. 转到设置 > 我的个人资料，并生成一个新的API令牌。


In [ ]:
%pip install llama-index-llms-predibase

In [ ]:
!pip install llama-index --quiet
!pip install predibase --quiet
!pip install sentence-transformers --quiet

In [ ]:
import os

os.environ["PREDIBASE_API_TOKEN"] = "{PREDIBASE_API_TOKEN}"
from llama_index.llms.predibase import PredibaseLLM

## 流程 1：直接查询Predibase LLM

在这个流程中，我们将直接查询Predibase LLM数据库，以获取所需的信息。


In [ ]:
# Predibase托管的微调适配器示例
llm = PredibaseLLM(
    model_name="mistral-7b",
    predibase_sdk_version=None,  # 可选参数（如果省略，默认为最新的Predibase SDK版本）
    adapter_id="e2e_nlg",  # adapter_id是可选的
    adapter_version=1,  # 可选参数（仅适用于Predibase）
    temperature=0.3,
    max_new_tokens=512,
)
# `model_name`参数是Predibase的“无服务器”base_model ID
# （请参阅https://docs.predibase.com/user-guide/inference/models以获取目录）。
# 您还可以选择指定托管在Predibase或HuggingFace上的微调适配器
# 对于Predibase托管的适配器，您还必须指定adapter_version

In [ ]:
# 基于HuggingFace托管的微调适配器示例
llm = PredibaseLLM(
    model_name="mistral-7b",
    predibase_sdk_version=None,  # 可选参数（如果省略，默认为最新的Predibase SDK版本）
    adapter_id="predibase/e2e_nlg",  # adapter_id是可选的
    temperature=0.3,
    max_new_tokens=512,
)
# `model_name`参数是Predibase的“无服务器”base_model ID
# （参见https://docs.predibase.com/user-guide/inference/models获取目录）。
# 您还可以选择指定托管在Predibase或HuggingFace上的微调适配器
# 对于Predibase托管的适配器，您还可以指定adapter_version（如果省略，假定为最新版本）

In [ ]:
result = llm.complete("Can you recommend me a nice dry white wine?")
print(result)

## 流程2：检索增强生成（RAG）与Predibase LLM

RAG是一种结合了检索和生成的模型，可以用于生成与给定查询相关的文本。Predibase LLM是一种基于语言的模型，用于生成与给定主题相关的文本。在这个流程中，我们将结合使用RAG和Predibase LLM，以实现检索增强生成的效果。


In [ ]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.core.embeddings import resolve_embed_model
from llama_index.core.node_parser import SentenceSplitter

#### 下载数据


In [ ]:
!mkdir -p 'data/paul_graham/'
!wget 'https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/paul_graham/paul_graham_essay.txt' -O 'data/paul_graham/paul_graham_essay.txt'

### 加载文档


In [ ]:
documents = SimpleDirectoryReader("./data/paul_graham/").load_data()

### 配置Predibase LLM


In [ ]:
# Predibase托管的经过微调的适配器
llm = PredibaseLLM(
    model_name="mistral-7b",
    predibase_sdk_version=None,  # 可选参数（如果省略，则默认为最新的Predibase SDK版本）
    adapter_id="e2e_nlg",  # adapter_id是可选的
    temperature=0.3,
    context_window=1024,
)

In [ ]:
# HuggingFace托管的微调适配器
llm = PredibaseLLM(
    model_name="mistral-7b",
    predibase_sdk_version=None,  # 可选参数（如果省略，默认为最新的Predibase SDK版本）
    adapter_id="predibase/e2e_nlg",  # adapter_id是可选的
    temperature=0.3,
    context_window=1024,
)

In [ ]:
embed_model = resolve_embed_model("local:BAAI/bge-small-en-v1.5")
splitter = SentenceSplitter(chunk_size=1024)

### 设置和查询索引


In [ ]:
index = VectorStoreIndex.from_documents(
    documents, transformations=[splitter], embed_model=embed_model
)
query_engine = index.as_query_engine(llm=llm)
response = query_engine.query("What did the author do growing up?")

In [ ]:
print(response)